In [1]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import datetime as dt
import pandas_ta as ta
import yfinance as yf
import copy
from pymongo import MongoClient
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
import time

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

load_dotenv()
cts_engine = create_engine(os.environ.get("CTS_URI"))
t2m_engine = create_engine(os.environ.get("T2M_URI"))


#### Chuẩn bị dữ liệu

##### Các hàm tính toán

In [2]:
#Đọc name map để chuyển đỏi các tên thành dạng full
name_map = pd.read_excel("../.xlsx/stock_classification.xlsx", sheet_name='name_map').drop(columns=['group', 'order'],axis=1)
name_map_dict = name_map.set_index('code')['full_name'].to_dict()

order_map = pd.read_excel("../.xlsx/stock_classification.xlsx", sheet_name='name_map').drop(columns=['group', 'full_name'],axis=1)
order_map_dict = order_map.set_index('code')['order'].to_dict()

group_map = pd.read_excel("../.xlsx/stock_classification.xlsx", sheet_name='name_map').drop(columns=['order', 'full_name'],axis=1)
group_map_dict = group_map.set_index('code')['group'].to_dict()

#Tạo các danh sách nhóm trong mỗi cách chia cổ phiếu
all_stock_key_list = ['all_stock']
industry_name_list = [key for key, value in group_map_dict.items() if value in ['A', 'B', 'C', 'D']]
industry_perform_list = [key for key, value in group_map_dict.items() if value == 'hs']
marketcap_group_list = [key for key, value in group_map_dict.items() if value == 'cap']

#Tạo danh danh key cho tổng tất cả các nhóm
group_stock_key_list = all_stock_key_list + industry_name_list + industry_perform_list + marketcap_group_list

In [3]:
def decode_data(file_path):
    # Đọc dữ liệu vào một numpy array
    data = np.fromfile(file_path, dtype=np.uint8)

    # Giả định kích thước mỗi bản ghi (có thể thay đổi tùy theo cấu trúc tệp thực tế)
    record_size = 32  # Giả định
    num_records = len(data) // record_size

    # Số lượng cột dữ liệu (bao gồm ngày, thời gian và các giá trị int32 còn lại)
    num_columns = record_size // 4  # Mỗi giá trị int32 chiếm 4 byte

    # Sử dụng numpy để cắt và giải mã dữ liệu hiệu quả hơn
    # Tạo một numpy array để chứa các giá trị int32 và float32
    raw_data = data.reshape(num_records, record_size // 4, 4)

    # Giải mã ngày và thời gian (int32) ở cột 0 và 1, các cột còn lại là float32
    int_data = raw_data[:, :2].view(np.int32)  # Giải mã int32 (2 cột)
    float_data = raw_data[:, 2:].view(np.float32)  # Giải mã float32 (các cột còn lại)

    # Kết hợp dữ liệu
    records = np.hstack((int_data, float_data))

    # Đảm bảo rằng dữ liệu là 2D
    records = records.reshape(num_records, num_columns)

    # Đảo ngược lại dữ liệu trước khi chuyển thành DataFrame
    records = records[::-1]

    # Chuyển đổi thành DataFrame và loại bỏ dòng đầu tiên
    df = pd.DataFrame(records, columns=[f"Col_{i}" for i in range(num_columns)])
    return df  # Loại bỏ đi dòng dữ liệu đầu tiên không cần thiết

def get_file_name_list(folder_path):
    file_name_list = []
    files = os.listdir(folder_path)
    for file in files:
        file_name_list.append(file[:-4])
    return file_name_list

def filter_market_file_name_list(file_name_list):
    filtered_list = [item for item in file_name_list if not (item.endswith('_AC') or item.endswith('_CC'))]
    return filtered_list

eod_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\stock"
eod_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\index"
eod_futures_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\futures"
itd_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\stock"
itd_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\index"
itd_futures_folder_path = "D:\\fireant_metakit\\AmiBroker\\Intraday\\futures"
nn_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\foreign"
td_stock_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\prop"
nntd_index_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\market"
other_folder_path = "D:\\fireant_metakit\\AmiBroker\\EOD\\other"

In [4]:
#Tạo dict map thời gian và số lượng cổ phiếu
period_map = pd.read_excel("../.xlsx/period_stock_list.xlsx", sheet_name='period_map')
period_map_dict = period_map.set_index('index').apply(lambda row: row.tolist(), axis=1).to_dict()

#Xoá đi quý hiện tại để chỉ tính toán tới quý trước đó
def get_quarter(name):
    now = datetime.now()
    year = now.year
    month = now.month
    if 1 <= month <= 3:
        quarter = "q1"
        previous_quarter = "q4"
        previous_year = year - 1
    elif 4 <= month <= 6:
        quarter = "q2"
        previous_quarter = "q1"
        previous_year = year
    elif 7 <= month <= 9:
        quarter = "q3"
        previous_quarter = "q2"
        previous_year = year
    else:
        quarter = "q4"
        previous_quarter = "q3"
        previous_year = year
    
    if name == 'current_quarter':
        return f'{quarter}_{year}'
    if name == 'previous_quarter':
        return f'{previous_quarter}_{previous_year}'

#Lấy ra list cổ phiếu của giai đoạn hiện tại
period_stock_list = pd.read_excel("../.xlsx/stock_classification.xlsx", sheet_name='period_stock_list')

current_quarter_stock_list = list(set(get_file_name_list(itd_stock_folder_path)) 
                                  & set(period_stock_list[get_quarter('current_quarter')].dropna().tolist()))

total_stock_list = list(set(get_file_name_list(itd_stock_folder_path)) & set(period_stock_list['all_stock'].dropna().tolist()))

#Lấy ra khoảng thời gian tính toán cho quý này và quý trước
calculate_time_span = [period_map_dict['q2_2020'][0], period_map_dict[get_quarter('current_quarter')][1]]
current_quarter_span = [period_map_dict[get_quarter('current_quarter')][0], period_map_dict[get_quarter('current_quarter')][1]]
previous_quarter_span = [period_map_dict[get_quarter('previous_quarter')][0], period_map_dict[get_quarter('previous_quarter')][1]]

##### Đọc dữ liệu EOD

In [5]:
def clean_stock_data(df_raw):
    #Lọc ra ra dữ liệu từ năm 2020
    df_raw = df_raw[df_raw['Col_0'] > 20200400]
    #Tạo cột cap cho cổ phiếu
    df_raw['cap'] = (df_raw['Col_5'] * df_raw['Col_7'])/1000000
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_1', 'Col_7'])
    #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'cap']

    return df_clean.reset_index(drop=True)

def clean_index_data(df_raw):
    #Lọc ra ra dữ liệu từ năm 2020
    df_raw = df_raw[df_raw['Col_0'] > 20200400]
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_1'])
    #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'option']
    #Điều chỉnh lại giá trị các cột
    df_clean['option'] = df_clean['option']/1000000000
		#Thêm cột phân loại index
    df_clean.insert(0, 'type', 'spot')
    
    return df_clean.reset_index(drop=True)

def clean_futures_data(df_raw):
    #Lọc ra ra dữ liệu từ năm 2020
    df_raw = df_raw[df_raw['Col_0'] > 20200400]
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_1'])
    #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'option']
		#Thêm cột phân loại index
    df_clean.insert(0, 'type', 'futures')
    
    return df_clean.reset_index(drop=True)

index_dict = {}
for ticker in get_file_name_list(eod_index_folder_path):
    temp_file_path = eod_index_folder_path + f'\\{ticker}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_index_data(temp_df_raw)
    temp_df_clean.insert(0, 'ticker', ticker)
    index_dict[ticker] = temp_df_clean
for ticker in get_file_name_list(eod_futures_folder_path):
    temp_file_path = eod_futures_folder_path + f'\\{ticker}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_futures_data(temp_df_raw)
    temp_df_clean.insert(0, 'ticker', ticker)
    index_dict[ticker] = temp_df_clean

stock_dict = {}
for ticker in total_stock_list:
    if ticker not in get_file_name_list(eod_stock_folder_path):
        pass
    else:
        temp_file_path = eod_stock_folder_path + f'\\{ticker}.dat'
        temp_df_raw = decode_data(temp_file_path)
        temp_df_clean = clean_stock_data(temp_df_raw)
        temp_df_clean.insert(0, 'ticker', ticker)
        stock_dict[ticker] = temp_df_clean


##### Đọc dữ liệu ITD

In [6]:
def clean_itd_data(df_raw):
    #Lọc ra đúng 1 ngày dữ liệu cuối cùng
    df_raw = df_raw[df_raw['Col_0'] == max(df_raw['Col_0'])]
    #Tạo cột date-time mới từ 2 cột date và time cũ
    df_raw['date'] = df_raw['Col_0'].astype(int).astype(str) + ' ' + df_raw['Col_1'].astype(int).astype(str)
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_0', 'Col_1', 'Col_7'])
    #Sắp xếp lại thứ tự các cột
    df_clean = df_clean[['date'] + [f"Col_{i}" for i in range(2, len(df_clean.columns)+1)]]
    # #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['date'] = pd.to_datetime(df_clean['date'], format='%Y%m%d %H%M%S')
    # #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    #Làm tròn khung thời gian tới 5 phút
    df_clean['date'] = df_clean['date'].dt.floor('5T')
    df_clean = df_clean.set_index("date").resample("5T", closed='right', label='right').agg({    
        "open": "first",  
        "high": "max",  
        "low": "min", 
        "close": "last",  
        "volume": "sum"   
    }).dropna().reset_index()

    return df_clean.sort_values(by="date", ascending=False).reset_index(drop=True)

itd_index_dict = {}
for index in get_file_name_list(itd_index_folder_path):
    temp_file_path = itd_index_folder_path + f'\\{index}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_itd_data(temp_df_raw)
    temp_df_clean.insert(0, 'ticker', index)
    itd_index_dict[index] = temp_df_clean
for index in get_file_name_list(itd_futures_folder_path):
    temp_file_path = itd_futures_folder_path + f'\\{index}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_itd_data(temp_df_raw)
    temp_df_clean.insert(0, 'ticker', index)
    itd_index_dict[index] = temp_df_clean

##### Đọc dữ liệu NNTD

In [7]:
def clean_nntd_index_data(df_raw, ticker):
    #Lọc ra ra dữ liệu từ năm 2020
    df_raw = df_raw[df_raw['Col_0'] > 20200400]
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_1', 'Col_3', 'Col_4'])
    #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'sell_volume', 'buy_volume', 'sell_value', 'buy_value']
    #Điều chỉnh lại giá trị các cột
    df_clean['buy_volume'] = df_clean['buy_volume']
    df_clean['sell_volume'] = -df_clean['sell_volume']
    df_clean['buy_value'] = df_clean['buy_value']/1000000000
    df_clean['sell_value'] = -df_clean['sell_value']/1000000000
    df_clean['net_volume'] = df_clean['buy_volume'] + df_clean['sell_volume']
    df_clean['net_value'] = df_clean['buy_value'] + df_clean['sell_value']

    if ticker in ['VN30F1M_NN', 'VN30F1M_TD', 'VN30F1Q_NN', 'VN30F1Q_TD', 'VN30F2M_NN', 'VN30F2M_TD', 'VN30F2Q_NN', 'VN30F2Q_TD']:
        df_clean['type'] = 'futures'
    else:
        df_clean['type'] = 'spot'

    df_clean['ticker'] = ticker

    return df_clean.reset_index(drop=True)

def clean_nntd_stock_data(df_raw):
    #Lọc ra ra dữ liệu từ năm 2020
    df_raw = df_raw[df_raw['Col_0'] > 20200400]
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_1', 'Col_3', 'Col_4'])
    #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'sell_volume', 'buy_volume', 'sell_value', 'buy_value']
    #Điều chỉnh lại giá trị các cột
    df_clean['buy_volume'] = df_clean['buy_volume']
    df_clean['sell_volume'] = -df_clean['sell_volume']
    df_clean['buy_value'] = df_clean['buy_value']/1000000000
    df_clean['sell_value'] = -df_clean['sell_value']/1000000000
    df_clean['net_volume'] = df_clean['buy_volume'] + df_clean['sell_volume']
    df_clean['net_value'] = df_clean['buy_value'] + df_clean['sell_value']
    
    return df_clean.reset_index(drop=True)

index_td_nn_dict = {}
for ticker in filter_market_file_name_list(get_file_name_list(nntd_index_folder_path)):
    temp_file_path = nntd_index_folder_path + f'\\{ticker}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_nntd_index_data(temp_df_raw, ticker)
    index_td_nn_dict[ticker] = temp_df_clean

stock_nn_dict = {}
for ticker in total_stock_list:
    if f'{ticker}_NN' not in get_file_name_list(nn_stock_folder_path):
        pass
    else:
        temp_file_path = nn_stock_folder_path + f'\\{ticker}_NN.dat'
        temp_df_raw = decode_data(temp_file_path)
        temp_df_clean = clean_nntd_stock_data(temp_df_raw)
        temp_df_clean.insert(0, 'ticker', f'{ticker}_NN')
        stock_nn_dict[f'{ticker}_NN'] = temp_df_clean

stock_td_dict = {}
for ticker in total_stock_list:
    if f'{ticker}_TD' not in get_file_name_list(td_stock_folder_path):
        pass
    else:
        temp_file_path = td_stock_folder_path + f'\\{ticker}_TD.dat'
        temp_df_raw = decode_data(temp_file_path)
        temp_df_clean = clean_nntd_stock_data(temp_df_raw)
        temp_df_clean.insert(0, 'ticker', f'{ticker}_TD')
        stock_td_dict[f'{ticker}_TD'] = temp_df_clean

##### Các biến thời gian

In [8]:
#Khởi tạo vnindex_series để xác định ngày hiện tại
vnindex_series = clean_index_data(decode_data(eod_index_folder_path + '\\VNINDEX.dat'))['date']

#Tạo date_series cho thời gian tính toán
date_series = pd.DataFrame(vnindex_series).rename(columns={0:'date'})
date_series = date_series[(date_series['date'] >= calculate_time_span[0]) & (date_series['date'] <= calculate_time_span[1])]

#Xác định ngày hiện tại
today = vnindex_series.iloc[0]

#Xác định thời gian hiện tại
current_time = itd_index_dict['UPINDEX']['date'].iloc[0]

#Khởi tạo time_series bao gồm tất cả khung thời gian của ngày hiện tại
time_series_list = []
time_series_list.extend(pd.date_range(start=f'{today} 09:00:00', end=f'{today} 11:25:00', freq='5T'))
time_series_list.extend(pd.date_range(start=f'{today} 13:00:00', end=f'{today} 14:55:00', freq='5T'))
time_series = pd.DataFrame(time_series_list).rename(columns={0:'date'})

#Điều chỉnh lại time_series bỏ đi các hàng thời gian chưa có dữ liệu
time_series = time_series.loc[time_series['date'].dt.time <= current_time.time()].sort_values('date', ascending=False).reset_index(drop=True)

#Khởi tạo khung thời gian bắt đầu từ 9h00 để vẽ các biểu đồ
itd_series = pd.DataFrame(time_series_list).rename(columns={0:'date'}).sort_values('date', ascending=False).reset_index(drop=True)

#Đọc file phân bổ thanh khoản trong phiên
itd_time_percent = pd.read_excel('../.xlsx/itd_time_percent.xlsx')
itd_time_percent = itd_time_percent.set_index("date").resample("5T", closed='right', label='right').agg({"percent": "last"}).dropna().reset_index()

#Chuyển đổi ngày thành này hôm nay
itd_time_percent['date'] = itd_time_percent['date'].apply(lambda x: current_time.replace(hour=x.hour, minute=x.minute, second=x.second))
#Khởi tạo hệ số thời gian
current_time_percent = itd_time_percent[itd_time_percent['date'] == current_time]['percent'].item()

##### Đọc dữ liệu other

In [9]:
def clean_other_data(df_raw, type):
    #Lọc ra ra dữ liệu từ năm 2020
    df_raw = df_raw[df_raw['Col_0'] > 20200400]
    #Xoá đi các cột khong sử dụng
    df_clean = df_raw.drop(columns=['Col_1'])
    #Chuyển đổi định dạng dữ liệu dang datetime
    df_clean['Col_0'] = pd.to_datetime(df_clean['Col_0'], format='%Y%m%d')
    #Đổi tên cột cho đúng
    df_clean.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'option']
		#Thêm cột phân loại index
    df_clean.insert(0, 'type', type)
    
    return df_clean.reset_index(drop=True)

def get_yfinance_df(ticker, symbol, type_name):
	temp_df = yf.download(symbol, start="2020-04-01", end=datetime.now().strftime('%Y-%m-%d'), progress=False, auto_adjust=True).reset_index()
	temp_df.columns = temp_df.columns.droplevel(1)
	temp_df.columns.name = None
	temp_df = temp_df.reset_index(drop=True)
	temp_df.columns = ['date','open','high','low','close','volume']
	temp_df['ticker'] = ticker
	temp_df['type'] = type_name
	return temp_df[['ticker','type','date','open','high','low','close','volume']]

other_dict = {}

#DXY và USDVND
other_dict['DXY'] = get_yfinance_df('DXY', "DX-Y.NYB", 'forex')
other_dict['USD_VND'] = get_yfinance_df('USD_VND', "USDVND=X", 'forex')

#Bạc và đồng
other_dict['SI=F'] = get_yfinance_df('SI=F', "SI=F", 'commodity')
other_dict['HG=F'] = get_yfinance_df('HG=F', "HG=F", 'commodity')

#Hai loại dâu mỏ và khí tự nhiên
other_dict['BZ=F'] = get_yfinance_df('BZ=F', "BZ=F", 'commodity')
other_dict['CL=F'] = get_yfinance_df('CL=F', "CL=F", 'commodity')
other_dict['NG=F'] = get_yfinance_df('NG=F', "NG=F", 'commodity')

#XAU USD
for ticker in ['^XAU']:
    temp_file_path = other_folder_path + f'\\{ticker}' + '\\USD.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_other_data(temp_df_raw, 'commodity')
    temp_df_clean.insert(0, 'ticker', ticker)
    other_dict[f'{ticker[1:]}_USD'] = temp_df_clean
    
for ticker in ['^SPX','^DJI','^IXIC','^NYA','^N225','^FTSE','^HIS','^SSEC','^STOXX50E','^SPX','^SPX']:
    temp_file_path = other_folder_path + f'\\{ticker}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_other_data(temp_df_raw, 'index')
    temp_df_clean.insert(0, 'ticker', ticker)
    other_dict[ticker] = temp_df_clean

#Các cặp tiền forex
for ticker in ['^AUD','^EUR','^GBP','^NZD']:
    temp_file_path = other_folder_path + f'\\{ticker}' + '\\USD.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_other_data(temp_df_raw, 'forex')
    temp_df_clean.insert(0, 'ticker', ticker)
    other_dict[f'{ticker[1:]}_USD'] = temp_df_clean
for ticker in get_file_name_list(f'{other_folder_path}\\^USD'):
    temp_file_path = other_folder_path+ '\\^USD' + f'\\{ticker}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_other_data(temp_df_raw, 'forex')
    temp_df_clean.insert(0, 'ticker', ticker)
    other_dict[f'USD_{ticker}'] = temp_df_clean
    
#Các cặp tiền crypto
for ticker in ['^BTC','^BCH','^LTC','^XRP','^ETH']:
    temp_file_path = other_folder_path + f'\\{ticker}' + '\\USD.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_other_data(temp_df_raw, 'crypto')
    temp_df_clean.insert(0, 'ticker', ticker)
    other_dict[f'{ticker[1:]}_USD'] = temp_df_clean
    
#Trái phiếu chính phủ
for ticker in ['^USBY10Y', '^USBY1Y', '^USBY5Y', '^VNBY10Y', '^VNBY1Y', '^VNBY5Y']:
    temp_file_path = other_folder_path + f'\\{ticker}.dat'
    temp_df_raw = decode_data(temp_file_path)
    temp_df_clean = clean_other_data(temp_df_raw, 'bond')
    temp_df_clean.insert(0, 'ticker', ticker)
    other_dict[f'{ticker[1:]}'] = temp_df_clean

for ticker, df in other_dict.items():
     other_dict[ticker] = df.sort_values('date', ascending=False).reset_index(drop=True)

##### ĐIều chỉnh dữ liệu

- Điều chỉnh dữ liệu EOD

In [10]:
#Bù dữ liệu vào những ngày cổ phiếu không có giao dịch
for ticker, df in stock_dict.items():
    temp_df = date_series.merge(df, on='date', how='left')
    temp_df[['open','high','low','close','cap']] = temp_df[['open','high','low','close','cap']].bfill()
    temp_df['ticker'] = temp_df['ticker'].fillna(ticker)
    temp_df['volume'] = temp_df['volume'].fillna(0)
    stock_dict[ticker] = temp_df

In [11]:
#Tính toán các đường trung bình và các đường MA
stock_dict = {k: v.sort_values(by=['date'], ascending=True).reset_index(drop=True) for k, v in stock_dict.items()}

stock_dict = {
    key: df.assign(
        high5=df['high'].rolling(window=5, min_periods=1).max(),
        low5=df['low'].rolling(window=5, min_periods=1).min(),
        high20=df['high'].rolling(window=20, min_periods=1).max(),
        low20=df['low'].rolling(window=20, min_periods=1).min(),
        high60=df['high'].rolling(window=60, min_periods=1).max(),
        low60=df['low'].rolling(window=60, min_periods=1).min(),
        high120=df['high'].rolling(window=120, min_periods=1).max(),
        low120=df['low'].rolling(window=120, min_periods=1).min(),
        high240=df['high'].rolling(window=240, min_periods=1).max(),
        low240=df['low'].rolling(window=240, min_periods=1).min(),
        high480=df['high'].rolling(window=480, min_periods=1).max(),
        low480=df['low'].rolling(window=480, min_periods=1).min(),

        ma5=df['close'].rolling(window=5, min_periods=1).mean(),
        ma5_V=df['volume'].rolling(window=5, min_periods=1).mean().shift(1),
    )
    for key, df in stock_dict.items()
}

stock_dict = {
    key: df.assign(
        trend_5p=(df['close'] > ((df['high5'] + df['low5'])/2).shift(1)).astype(int),
        trend_20p=(df['close'] > ((df['high20'] + df['low20'])/2).shift(1)).astype(int),
        trend_60p=(df['close'] > ((df['high60'] + df['low60'])/2).shift(1)).astype(int),
        trend_120p=(df['close'] > ((df['high120'] + df['low120'])/2).shift(1)).astype(int),
        trend_240p=(df['close'] > ((df['high240'] + df['low240'])/2).shift(1)).astype(int),
        trend_480p=(df['close'] > ((df['high480'] + df['low480'])/2).shift(1)).astype(int)
    )
    for key, df in stock_dict.items()
}

stock_dict = {k: v.sort_values(by=['date'], ascending=False).reset_index(drop=True) for k, v in stock_dict.items()}

In [12]:
#Chuyển đổi thời gian trong period_map_dict sang pd.Timestamp để có thể so sánh với ngày trong df
period_timestamp_map_dict = {}
for key, value in period_map_dict.items():
    period_timestamp_map_dict[key] = (pd.Timestamp(value[0]), pd.Timestamp(value[1]), value[2])

#Thêm các cột count và period vào df
for ticker, df in stock_dict.items():
    # Đảm bảo cột 'date' là kiểu Timestamp
    df['date'] = pd.to_datetime(df['date'])
    # Gán 'quarter' dựa trên việc kiểm tra khoảng thời gian
    df['quarter'] = df['date'].apply(lambda x: next((key for key, value in period_timestamp_map_dict.items() if value[0] <= x <= value[1]), None))
    # Gán 'count' dựa trên 'period'
    df['stock_count'] = df['quarter'].apply(lambda x: period_timestamp_map_dict[x][2] if x else None)
    # Cập nhật lại DataFrame vào stock_dict
    stock_dict[ticker] = df

#Tính hệ số thanh khoản và sửa các giá trị 0 và inf
for df in stock_dict.values():
    df['vol_ratio'] = df['volume'] / df['ma5_V']
    # Tính giá trị đầu tiên của vol_ratio với current_time_percent
    df['vol_ratio'].iloc[0] = df['volume'].iloc[0] / (df['ma5_V'].iloc[0] * current_time_percent)
    # Thay thế các giá trị inf trong vol_ratio bằng 0
    df['vol_ratio'].replace([np.inf, -np.inf], 0, inplace=True)

- Điều chỉnh dữ liệu ITD

In [13]:
#Bù lại những dòng thời gian bị thiếu của index sàn HSX
for ticker, df in itd_index_dict.items():
    df = time_series.merge(df, on='date', how='left').sort_values('date', ascending=False)
    
    #Thêm giá trị mở cửa lúc 9h cho các cổ phiếu HSX
    if pd.isna(df['ticker'].iloc[-1]):
        df.loc[df.index[-1], ['open', 'high', 'low', 'close']] = index_dict[ticker].at[1, 'close']

    df[['open','high','low','close']] = df[['open','high','low','close']].bfill()
    df['volume'] = df['volume'].fillna(0)
    df['ticker'] = ticker

    #Điều chỉnh lại giá cuối của itd cho giống với eod
    df.loc[0, 'close'] = index_dict[ticker].at[0, 'close']
    df.loc[0, 'low'] = index_dict[ticker].at[0, 'low']
    df.loc[0, 'high'] = index_dict[ticker].at[0, 'high']

    itd_index_dict[ticker] = df

- Điều chỉnh dữ liệu NNTD

In [14]:
#Bù dữ liệu vào những ngày index không có giao dịch
for ticker, df in index_td_nn_dict.items():
    temp_df = date_series.merge(df, on='date', how='left')
    temp_df[['sell_volume','buy_volume','sell_value','buy_value','net_volume','net_value']] = temp_df[['sell_volume','buy_volume','sell_value','buy_value','net_volume','net_value']].fillna(0)
    temp_df['ticker'] = temp_df['ticker'].fillna(ticker)
    temp_df['type'] = temp_df['type'].bfill()
    temp_df['type'] = temp_df['type'].ffill()
    index_td_nn_dict[ticker] = temp_df.iloc[:20]

#Bù dữ liệu vào những ngày cổ phiếu không có giao dịch
for ticker, df in stock_td_dict.items():
    temp_df = date_series.merge(df, on='date', how='left')
    temp_df[['sell_volume','buy_volume','sell_value','buy_value','net_volume','net_value']] = temp_df[['sell_volume','buy_volume','sell_value','buy_value','net_volume','net_value']].fillna(0)
    temp_df['ticker'] = temp_df['ticker'].fillna(ticker)
    stock_td_dict[ticker] = temp_df.iloc[:20]

#Bù dữ liệu vào những ngày cổ phiếu không có giao dịch
for ticker, df in stock_nn_dict.items():
    temp_df = date_series.merge(df, on='date', how='left')
    temp_df[['sell_volume','buy_volume','sell_value','buy_value','net_volume','net_value']] = temp_df[['sell_volume','buy_volume','sell_value','buy_value','net_volume','net_value']].fillna(0)
    temp_df['ticker'] = temp_df['ticker'].fillna(ticker)
    stock_nn_dict[ticker] = temp_df.iloc[:20]

#### Điểm dòng tiền

##### Phân nhóm cổ phiếu

In [15]:
#Lấy danh sách phân loại cổ phiếu
stock_classification_df = pd.read_excel("../.xlsx/stock_classification.xlsx", sheet_name='stock_classification').rename(columns={'stock':'ticker'})
stock_classification_df = stock_classification_df[stock_classification_df['ticker'].isin(total_stock_list)].reset_index(drop=True)
current_quarter_classification_df = stock_classification_df[stock_classification_df['ticker'].isin(current_quarter_stock_list)].reset_index(drop=True)

price_arr = []
cap_arr = []
for ticker in current_quarter_classification_df['ticker']:
    df = stock_dict[ticker].copy()
    df = df[df['date'] >= current_quarter_span[0]]
    price_arr.append(df['close'].iloc[-1].item())
    cap_arr.append(df['cap'].iloc[-1].item())

vonhoa_classification_df = current_quarter_classification_df.copy()
vonhoa_classification_df['price'] = price_arr
vonhoa_classification_df['cap'] = cap_arr

cap_coef = sum(cap_arr)/10000
vonhoa_classification_df['marketcap_group'] = vonhoa_classification_df.apply(lambda x:
'small' if ((x['cap']>cap_coef) & (x['cap']<10*cap_coef)) | 
        ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']<10)) 
        else
('mid' if ((x['cap']>=10*cap_coef) & (x['cap']<20*cap_coef) & (x['price']>=10)) | 
        ((x['cap']>=20*cap_coef) & (x['cap']<100*cap_coef))
        else
('large' if x['cap']>=100*cap_coef
        else 'penny'
)), axis=1)

current_quarter_classification_df = pd.concat([current_quarter_classification_df, vonhoa_classification_df['marketcap_group']], axis=1)

#Lấy phân loại của quý này từ dữ liệu lịch sử, các cổ phiếu không thuộc danh sách theo dõi quý này sẽ không được phân loại và nhận giá trị Nan
stock_classification_df['marketcap_group'] = stock_classification_df['ticker'].map(current_quarter_classification_df.set_index('ticker')['marketcap_group'])

In [16]:
# Convert DataFrame columns to dictionaries for quick access
stock_by_industry = current_quarter_classification_df.set_index('ticker')['industry_name'].to_dict()
stock_by_perform = current_quarter_classification_df.set_index('ticker')['industry_perform'].to_dict()
stock_by_marketcap = current_quarter_classification_df.set_index('ticker')['marketcap_group'].to_dict()

# Initialize dictionaries
eod_all_stock = {}
eod_industry_name = {}
eod_industry_perform = {}
eod_marketcap_group = {}

# Function to create mappings based on category
def create_mapping(stock_dict, category_dict):
    category_map = {}
    for category, stocks in category_dict.items():
        category_map[category] = {ticker: stock_dict[ticker] for ticker in stocks if ticker in stock_dict}
    return category_map

# Precompute unique categories and relevant stocks
unique_industries = np.unique(list(stock_by_industry.values()))
unique_performs = np.unique(list(stock_by_perform.values()))
unique_marketcaps = ['large', 'mid', 'small', 'penny']

# Mapping for all_stock
eod_all_stock['all_stock'] = {ticker: value for ticker, value in stock_dict.items() if ticker in current_quarter_stock_list}

# Mapping for industry
for industry in unique_industries:
    relevant_stocks = [ticker for ticker, ind in stock_by_industry.items() if (ind == industry) and (ticker in current_quarter_stock_list)]
    eod_industry_name[industry] = {ticker: stock_dict[ticker] for ticker in relevant_stocks if ticker in stock_dict}

# Mapping for performance
for performance in unique_performs:
    relevant_stocks = [ticker for ticker, perf in stock_by_perform.items() if (perf == performance) and (ticker in current_quarter_stock_list)]
    eod_industry_perform[performance] = {ticker: stock_dict[ticker] for ticker in relevant_stocks if ticker in stock_dict}

# Mapping for marketcap
for marketcap in unique_marketcaps:
    relevant_stocks = [ticker for ticker, mcap in stock_by_marketcap.items() if (mcap == marketcap) and (ticker in current_quarter_stock_list)]
    eod_marketcap_group[marketcap] = {ticker: stock_dict[ticker] for ticker in relevant_stocks if ticker in stock_dict}


##### Điểm dòng tiền cổ phiếu

In [17]:
def score_calculation(row):
    try:
        # Tính toán giá trị điểm số
        result = (((row['close'] - row['low']) - (row['high'] - row['close'])) / (row['high'] - row['low']) *
                  abs((row['close'] - row['close_prev'])) / row['close_prev'] *
                  (row['volume'] * row['close']) / (row['ma5_prev'] * row['ma5_V'])) * 100 \
                  + ((row['close'] - row['ma5_prev']) / row['ma5_prev']) / 100
        
        # Kiểm tra nếu kết quả là inf, trả về 0 nếu đúng
        if np.isinf(result):
            # Xử lý khi xảy ra lỗi chia cho 0, trả về giá trị tính toán thêm
            return ((row['close'] - row['ma5_prev']) / row['ma5_prev']) / 100
        return result
    except ZeroDivisionError:
        # Xử lý khi xảy ra lỗi chia cho 0, trả về giá trị tính toán thêm
        return ((row['close'] - row['ma5_prev']) / row['ma5_prev']) / 100

def score_calculation_t0(row):
    try:
        # Tính toán giá trị điểm số
        result = (((row['close'] - row['low']) - (row['high'] - row['close'])) / (row['high'] - row['low']) *
                  abs((row['close'] - row['close_prev'])) / row['close_prev'] *
                  (row['volume'] * row['close']) / (row['ma5_prev'] * row['ma5_V'] * current_time_percent)) * 100 \
                  + ((row['close'] - row['ma5_prev']) / row['ma5_prev']) / 100
        
        # Kiểm tra nếu kết quả là inf hoặc NaN
        if np.isinf(result):
            # Xử lý khi xảy ra lỗi chia cho 0, trả về giá trị tính toán thêm
            return ((row['close'] - row['ma5_prev']) / row['ma5_prev']) / 100
        elif np.isnan(result):
            return ((row['close'] - row['ma5_prev']) / row['ma5_prev']) / 100
        
        return result
    except ZeroDivisionError:
        # Xử lý khi xảy ra lỗi chia cho 0, trả về giá trị tính toán thêm
        return ((row['close'] - row['ma5_prev']) / row['ma5_prev']) / 100

In [18]:
#Tính điểm dòng tiền cho từng cổ phiếu
stock_score_dict = {}
for ticker in stock_dict.keys():

    #Lọc ra các cột cần sử dụng và chỉ lấy quý này và quý trước gần nhất để tính
    temp_df = stock_dict[ticker][['ticker', 'date', 'quarter', 'stock_count', 'open', 'high', 'low', 'close', 'volume', 'vol_ratio', 'ma5', 'ma5_V']]
    temp_df = temp_df[temp_df['date']>=previous_quarter_span[0]]

    #Tính điểm dòng tiền t0 và t5
    temp_df['ma5_prev'] = temp_df['ma5'].shift(-1)
    temp_df['close_prev'] = temp_df['close'].shift(-1)

    temp_df['t0_score'] = temp_df.apply(score_calculation, axis=1)
    temp_df['t0_score'].iloc[0] = score_calculation_t0(temp_df.iloc[0])

    temp_df['t5_score'] = temp_df['t0_score'][::-1].rolling(window=5, min_periods=1).mean()[::-1]
    temp_df['price_change'] = temp_df['close'][::-1].pct_change()[::-1]
    temp_df['value_change'] = temp_df['close'][::-1].diff()[::-1]

    #Gán lại temp_df cho dict
    stock_score_dict[ticker] = temp_df

#Tính xếp hạng cho cổ phiếu
t0_ranking_df = date_series.copy()
t5_ranking_df = date_series.copy()
for ticker in stock_score_dict.keys():
    t0_ranking_df[ticker] = stock_score_dict[ticker]['t0_score']
    t0_ranking_df.fillna(0, inplace=True)
    t5_ranking_df[ticker] = stock_score_dict[ticker]['t5_score']
    t5_ranking_df.fillna(0, inplace=True)
t0_ranking_df = t0_ranking_df.iloc[:,1:].rank(ascending=False, method='min', axis=1)
t5_ranking_df = t5_ranking_df.iloc[:,1:].rank(ascending=False, method='min', axis=1)

#Ghép xếp hạng vào bảng thông tin cổ phiếu
for ticker, df in stock_score_dict.items():
    df['rank_t0'] = t0_ranking_df[ticker]
    df['rank_t5'] = t5_ranking_df[ticker]
    
    #Check xem xếp hạng T0 nằm trong top 10% hay không
    df['top_check'] = df.apply(lambda x: 1 if x['rank_t0'] <= x['stock_count']*0.1 else 0, axis=1)

    #Tính số phiên lọt top 10% trong 20 phiên
    df['top_count'] = df['top_check'][::-1].rolling(window=20).sum()[::-1]

    stock_score_dict[ticker] = df

##### Điểm dòng tiền nhóm

In [19]:
#Chỉnh sửa lại điểm dòng tiền t0 cho từng cổ phiếu với tác động của độ rộng từng nhóm
def adjust_score_by_breath(t0_score, ratio_column):
    adjusted_score = []
    for score, ratio in zip(t0_score, ratio_column):
        if score >= 0:
            adjusted_score.append(score*ratio)
        else:
            adjusted_score.append(score*(1-ratio))
    return adjusted_score

#Hàm điều chỉnh điểm dòng tiền của cổ phiếu tránh sự đột biến khi đóng góp vào nhóm chung
def adjust_score_for_smooth(row, column_name, max_percent, mark):
    origin_score = row[column_name]
    
    if abs(origin_score) > row['total'] * max_percent:

        sum_abs = row['total'] - abs(row[column_name])
        fixed_score = sum_abs / (1 - max_percent) - sum_abs

        if origin_score >= 0:
            return fixed_score
        else:
            return -fixed_score
    else:
        mark[0] = 0
        return origin_score

#Áp dụng hàm điều chỉnh điểm phía trên vào các nhóm cổ phiếu, việc này lặp lại nhiều lần cho tới khi triệt tiêu sự đột biến
def apply_smooth_score(score_dict, group_type):
    if group_type == 'all_stock':
        key_list = all_stock_key_list
    elif group_type == 'industry_perform':
        key_list = [key for key, value in group_map_dict.items() if value == 'hs']
    elif group_type == 'marketcap_group':
        key_list = [key for key, value in group_map_dict.items() if value == 'cap'] 
    elif group_type == 'industry_name':
        key_list = [key for key, value in group_map_dict.items() if value in industry_perform_list]

    for key in key_list:
        score_df = date_series.copy()
        if group_type == 'all_stock':
            stock_list = current_quarter_stock_list
        else:
            stock_list = [ticker for ticker in current_quarter_classification_df[current_quarter_classification_df[group_type]==key]['ticker'].dropna().tolist()]
        for ticker in stock_list:
            try: score_df[ticker] = score_dict[ticker][f't0_score']
            except: pass

        max_percent = max(0.1, min(5*(1/len(stock_list)), 0.5))
        score_df['total'] = score_df.iloc[:, 1:].abs().sum(axis=1)

        mark = [1]
        while True:
            if mark[0] == 1:
                for ticker in stock_list:
                    if ticker in score_dict.keys():
                        score_df[ticker] = score_df.iloc[:, 1:].apply(adjust_score_for_smooth, axis=1, args=(ticker, max_percent, mark))
            if mark[0] == 0: break

        for ticker in score_dict.keys():
            if ticker in current_quarter_stock_list:
                if ticker in stock_list:
                    score_dict[ticker][f't0_{group_type}'] = score_df[ticker]
            else:
                score_dict[ticker][f't0_{group_type}'] = 0

In [20]:
#Thêm các cột dòng tiền đóng góp vào các nhóm cổ phiếu vào các dict period (đã loại bỏ đột biến)
for group_type in ['all_stock','industry_name','industry_perform','marketcap_group']:
    apply_smooth_score(stock_score_dict, group_type)

In [21]:
#Tính độ rộng cho từng phiên phục vụ cho việc điều chỉnh điểm dòng tiền
temp_df = date_series.copy()
for ticker, df in stock_score_dict.items():
    temp_df[ticker] = stock_score_dict[ticker]['t0_score']
temp_df.iloc[:,1:] = temp_df.iloc[:,1:].applymap(lambda x: 1 if x > 0 else 0)

eod_market_breath = date_series.copy()

industry_name_breadth_dict = {}
for key in eod_industry_name.keys():
    stock_list = current_quarter_classification_df[current_quarter_classification_df['industry_name']==key]['ticker'].tolist()
    industry_name_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    eod_market_breath[key] = industry_name_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

industry_perform_breadth_dict = {}
for key in eod_industry_perform.keys():
    stock_list = current_quarter_classification_df[current_quarter_classification_df['industry_perform']==key]['ticker'].tolist()
    industry_perform_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    eod_market_breath[key] = industry_perform_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

marketcap_group_breadth_dict = {}
for key in eod_marketcap_group.keys():
    stock_list = current_quarter_classification_df[current_quarter_classification_df['marketcap_group']==key]['ticker'].tolist()
    marketcap_group_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    eod_market_breath[key] = marketcap_group_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

all_stock_breadth_dict = {}
for key in eod_all_stock.keys():
    stock_list = current_quarter_classification_df['ticker'].tolist()
    all_stock_breadth_dict[key] = temp_df[['date'] + [columns for columns in stock_list]]
    eod_market_breath[key] = all_stock_breadth_dict[key].iloc[:,1:].sum(axis=1)/len(stock_list)

#Chỉnh sửa lại điểm dòng tiền t0 cho từng cổ phiếu với tác động của độ rộng từng nhóm
for ticker, df in stock_score_dict.items():
    if ticker in current_quarter_stock_list:
        name_of_industry_name = current_quarter_classification_df[current_quarter_classification_df['ticker']==ticker]['industry_name'].item()
        name_of_industry_perform = current_quarter_classification_df[current_quarter_classification_df['ticker']==ticker]['industry_perform'].item()
        name_of_marketcap_group = current_quarter_classification_df[current_quarter_classification_df['ticker']==ticker]['marketcap_group'].item()

        df[f't0_industry_name'] = adjust_score_by_breath(df['t0_industry_name'], eod_market_breath[name_of_industry_name])
        df[f't0_industry_perform'] = adjust_score_by_breath(df['t0_industry_perform'], eod_market_breath[name_of_industry_perform])
        df[f't0_marketcap_group'] = adjust_score_by_breath(df['t0_marketcap_group'], eod_market_breath[name_of_marketcap_group])
        df[f't0_all_stock'] = adjust_score_by_breath(df['t0_all_stock'], eod_market_breath['all_stock'])

- Bảng điểm dòng tiền nhóm

In [22]:
#Tạo bảng dữ liệu điểm dòng tiền cho các nhóm cổ phiếu
group_score_df = date_series.copy()

#Thêm cột điểm dòng tiền toàn bộ cổ phiếu
for nganh in eod_all_stock.keys():
    score_df = date_series.copy()
    for ticker in current_quarter_classification_df['ticker']:
        score_df[ticker] = stock_score_dict[ticker]['t0_all_stock']
    score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
    group_score_df[nganh] = score_df['total']

#Thêm các cột điểm dòng tiền ngành
eod_industry_name_score_df = date_series.copy()
for nganh in eod_industry_name.keys():
    score_df = date_series.copy()
    for ticker in current_quarter_classification_df[current_quarter_classification_df['industry_name']==nganh]['ticker']:
        score_df[ticker] = stock_score_dict[ticker]['t0_industry_name']
    score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
    group_score_df[nganh] = score_df['total']
    if nganh == 'bao_hiem': temp_df = score_df.copy()

#Thêm các cột điểm dòng tiền nhóm hiệu suất
eod_industry_perform_score_df = date_series.copy()
for group in eod_industry_perform.keys():
    score_df = date_series.copy()
    for ticker in current_quarter_classification_df[current_quarter_classification_df['industry_perform']==group]['ticker']:
        score_df[ticker] = stock_score_dict[ticker]['t0_industry_perform']
    score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
    group_score_df[group] = score_df['total']

#Thêm các cột điểm dòng tiền nhóm vốn hoá
eod_marketcap_group_score_df = date_series.copy()
for marketcap in eod_marketcap_group.keys():
    score_df = date_series.copy()
    for ticker in current_quarter_classification_df[current_quarter_classification_df['marketcap_group']==marketcap]['ticker']:
        score_df[ticker] = stock_score_dict[ticker]['t0_marketcap_group']
    score_df['total'] = score_df.iloc[:, 1:].mean(axis=1)
    group_score_df[marketcap] = score_df['total']

- Bảng xếp hạng nhóm

In [23]:
#Tạo bảng xếp hạng cho các nhóm cổ phiếu
def create_ranking_df(score_df):
    socre_dict = {}
    for group in score_df.columns[1:]:
        socre_dict[group] = date_series.copy()
        socre_dict[group]['t0_score'] = score_df[group]
        socre_dict[group]['t5_score'] = socre_dict[group]['t0_score'][::-1].rolling(window=5).mean()[::-1]

    ranking_score = date_series.copy()
    for group in socre_dict.keys():
        ranking_score[group] = socre_dict[group]['t5_score']
        ranking_score.fillna(0,inplace=True)

    ranking_df = date_series.copy()
    for group in socre_dict.keys():
        ranking_df[group] = 0

    for i in range(len(date_series.copy())):
        ranking_df.iloc[i, 1:] = ranking_score.iloc[i, 1:].rank(ascending=False, method='min')

    return ranking_df

industry_name_ranking_df = create_ranking_df(group_score_df[['date'] + industry_name_list])
industry_perform_ranking_df = create_ranking_df(group_score_df[['date'] + industry_perform_list])
marketcap_group_ranking_df = create_ranking_df(group_score_df[['date'] + marketcap_group_list])
group_score_ranking_df = industry_name_ranking_df.merge(industry_perform_ranking_df, on='date', how='left').merge(marketcap_group_ranking_df, on='date', how='left')

#Thêm cột xếp hạng cho nhóm 'all_stock' là 1 để khớp dữ liệu bảng
group_score_ranking_df['all_stock'] = 1

- Bảng hệ số thanh khoản nhóm

In [24]:
#Tạo dict chứa thanh khoản trung bình của mỗi nhóm cổ phiếu trong các giai đoạn
group_volume_df = date_series.copy()

for name in all_stock_key_list:
    temp_volume_df = date_series.copy()
    for stock, df in eod_all_stock[name].items():
        temp_volume_df[stock] = df['volume']
    temp_volume_df['volume'] = temp_volume_df.iloc[:, 1:].sum(axis=1)
    group_volume_df[name] = temp_volume_df['volume']

for name in industry_name_list:
    temp_volume_df = date_series.copy()
    for stock, df in eod_industry_name[name].items():
        temp_volume_df[stock] = df['volume']
    temp_volume_df['volume'] = temp_volume_df.iloc[:, 1:].sum(axis=1)
    group_volume_df[name] = temp_volume_df['volume']

for name in industry_perform_list:
    temp_volume_df = date_series.copy()
    for stock, df in eod_industry_perform[name].items():
        temp_volume_df[stock] = df['volume']
    temp_volume_df['volume'] = temp_volume_df.iloc[:, 1:].sum(axis=1)
    group_volume_df[name] = temp_volume_df['volume']

for name in marketcap_group_list:
    temp_volume_df = date_series.copy()
    for stock, df in eod_marketcap_group[name].items():
        temp_volume_df[stock] = df['volume']
    temp_volume_df['volume'] = temp_volume_df.iloc[:, 1:].sum(axis=1)
    group_volume_df[name] = temp_volume_df['volume']

#Tạo bảng hệ số thanh khoản để lấy phiên cuối
group_vol_ratio_df = date_series.copy()
temp_df = date_series.copy()
for key in group_stock_key_list:
    temp_df[key] = group_volume_df[key][::-1].rolling(window=5).mean()[::-1].shift(-1)
    group_vol_ratio_df[key] = group_volume_df[key]/temp_df[key]
    group_vol_ratio_df[key].iloc[0] = group_volume_df[key].iloc[0]/(temp_df[key].iloc[0])


#### Tính toán các bảng cần thiết

##### Bảng cho báo cáo hàng ngày

In [25]:
for ticker, df in index_dict.items():
    df['pct_change'] = df['close'][::-1].pct_change()[::-1]
    df['value_change'] = df['close'][::-1].diff()[::-1]
    df['pct_5'] = df['close'][::-1].rolling(window=5).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[-1])[::-1]
    df['pct_20'] = df['close'][::-1].rolling(window=20).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[-1])[::-1]
    df['pct_60'] = df['close'][::-1].rolling(window=60).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[-1])[::-1]

for ticker, df in other_dict.items():
    df['pct_change'] = df['close'][::-1].pct_change()[::-1]
    df['value_change'] = df['close'][::-1].diff()[::-1]
    df['pct_5'] = df['close'][::-1].rolling(window=5).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[-1])[::-1]
    df['pct_20'] = df['close'][::-1].rolling(window=20).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[-1])[::-1]
    df['pct_60'] = df['close'][::-1].rolling(window=60).apply(lambda x: (x.iloc[-1] - x.iloc[0])/x.iloc[-1])[::-1]

data_1 = {}
for ticker, df in index_dict.items():
    data_1[ticker] = df[['close','volume','pct_change','value_change','pct_5','pct_20','pct_60']].iloc[0]

for ticker, df in other_dict.items():
    data_1[ticker] = df[['close','volume','pct_change','value_change','pct_5','pct_20','pct_60']].iloc[0]

data_1 = pd.DataFrame.from_dict(data_1, orient='index').reset_index()

In [26]:
week_groupby = index_dict['VN30F1M'].set_index("date").resample("W", closed='right', label='right').agg({    
    "open": "first",  
    "high": "max",  
    "low": "min", 
    "close": "last",  
    "volume": "sum"   
}).dropna().sort_values('date', ascending=False).reset_index().iloc[0]

month_groupby = index_dict['VN30F1M'].set_index("date").resample("M", closed='right', label='right').agg({    
    "open": "first",  
    "high": "max",  
    "low": "min", 
    "close": "last",  
    "volume": "sum"   
}).dropna().sort_values('date', ascending=False).reset_index().iloc[0]

data_2 = {}
data_2['oi'] = index_dict['VN30F1M'].iloc[0]['option']
data_2['volume'] = index_dict['VN30F1M'].iloc[0]['volume']
data_2['basis'] = index_dict['VN30F1M'].iloc[0]['close'] - index_dict['VN30'].iloc[0]['close']
data_2['close'] = index_dict['VN30F1M'].iloc[0]['close']
data_2['open_month'] = month_groupby['open']
data_2['open_week'] = week_groupby['open']
data_2['highest_week'] = week_groupby['high']
data_2['lowest_week'] = week_groupby['low']
data_2['balanced_week'] = (week_groupby['high'] + week_groupby['low'])/2

data_2 = pd.DataFrame.from_dict(data_2, orient='index', columns=['value']).reset_index()

In [27]:
data_3 = pd.read_sql("SELECT * FROM market_price_breath_df", con=t2m_engine).drop(columns='index')
data_4 = pd.read_sql("SELECT * FROM group_eod_score_liquidity_df", con=t2m_engine).drop(columns='index')
data_5 = pd.DataFrame([pd.read_sql("SELECT * FROM market_sentiment_df", con=t2m_engine).dropna()[['last_ratio','last_sentiment']].iloc[0]])
data_6 = pd.read_sql("SELECT * FROM market_top_stock_df", con=t2m_engine).drop(columns='index')

data_7 = pd.read_sql("SELECT * FROM market_index_card_df", con=t2m_engine).drop(columns='index')
data_7 = data_7[data_7['stock'].isin(['VNINDEX','VN30','VN30F1M'])]
data_7['value_traded'] = [index_dict['VN30'].iloc[0]['option'], 
                          index_dict['VNINDEX'].iloc[0]['option'], 
                          index_dict['VN30F1M'].iloc[0]['close']*100000*index_dict['VN30F1M'].iloc[0]['volume']/1000000000]

data_8 = pd.read_sql("SELECT * FROM group_ms_chart_df", con=t2m_engine).drop(columns='index')
data_8 = data_8[data_8['name']=='Thị trường'].iloc[:60].drop(columns=['name'])
data_8.columns = ['Ngày', 'Tuần', 'Tháng', 'Quý', 'Bán niên', '1 Năm', '2 Năm']

data_9 = pd.read_sql("SELECT * FROM nn_td_20p_df", con=t2m_engine).drop(columns='index')
data_9 = data_9[data_9['id']=='HSX'].sort_values('date')

data_10 = pd.read_sql("SELECT * FROM nn_td_buy_sell_df", con=t2m_engine).drop(columns='index')
data_10 = data_10[data_10['id']=='HSX'].iloc[:,:5]

In [28]:
data_11 = pd.DataFrame([
    index_td_nn_dict['VN30F1M_NN'].iloc[0],
    index_td_nn_dict['VN30F1M_TD'].iloc[0],
    index_td_nn_dict['VN30F2M_NN'].iloc[0],
    index_td_nn_dict['VN30F2M_TD'].iloc[0]
])

data_11['luy_ke'] = [index_td_nn_dict['VN30F1M_NN']['net_volume'].iloc[:20][::-1].cumsum()[::-1].iloc[0],
                        index_td_nn_dict['VN30F1M_TD']['net_volume'].iloc[:20][::-1].cumsum()[::-1].iloc[0],
                        index_td_nn_dict['VN30F2M_NN']['net_volume'].iloc[:20][::-1].cumsum()[::-1].iloc[0],
                        index_td_nn_dict['VN30F2M_TD']['net_volume'].iloc[:20][::-1].cumsum()[::-1].iloc[0]
                    ]

data_12 = index_td_nn_dict['VN30F1M_NN'][['date','net_volume']].iloc[:20].rename(columns={'net_volume':'NN'})
data_12['TD'] = index_td_nn_dict['VN30F1M_TD']['net_volume'].iloc[:20]

In [29]:
data_13 = itd_series.copy().set_index('date')
for ticker, df in itd_index_dict.items():
    df_copy = df.set_index('date')
    data_13[f'{ticker}_close'] = df_copy['close']
    data_13[f'{ticker}_vol'] = df_copy['volume']

data_13 = data_13.reset_index().sort_values('date')

In [30]:
def create_day_series(year=2024, start_month=1, start_day=1, num_days=365):
    start_date = datetime(year, start_month, start_day)
    days = []
    
    for i in range(num_days):
        date = start_date + timedelta(days=i)
        formatted_date = date.strftime("%d-%b")
        days.append(formatted_date)
    
    return pd.Series(days, name='days')

# Tạo Series mới
data_14 = pd.DataFrame(create_day_series(num_days=366)).set_index('days')

# Tạo các năm
year_list = sorted(date_series['date'].dt.year.unique().tolist(), reverse=True)[:3]
year_dict = {}
for year in year_list:
    start_date = datetime(year, 1, 1).strftime('%Y-%m-%d')
    end_date = datetime(year, 12, 31).strftime('%Y-%m-%d')
    year_dict[year] = {'start': start_date,'end': end_date}

temp_ticker_dict = {}
for ticker in ['DXY', 'XAU_USD', 'BCH_USD', '^DJI', 'CL=F']:
    temp_ticker_dict[ticker] = other_dict[ticker].copy()
temp_ticker_dict['VNINDEX'] = index_dict['VNINDEX'].copy()

for ticker, df in temp_ticker_dict.items():
    for year, time_range in year_dict.items():
        temp_df = df[(df['date'] >= time_range['start']) & (df['date'] <= time_range['end'])][['date', 'close']]
        temp_df['pct_change'] = temp_df['close'][::-1].pct_change()[::-1].fillna(0)
        temp_df['cumsum'] = temp_df['pct_change'][::-1].cumsum()[::-1].fillna(0)
        temp_df['date'] = temp_df['date'].apply(lambda x: x.strftime("%d-%b"))
        temp_df = temp_df.set_index('date')
        temp_df = temp_df[~temp_df.index.duplicated(keep='first')]  
        data_14[f'{year}_{ticker}'] = temp_df['cumsum']

data_14 = data_14.bfill().reset_index()

#### Lưu vào SQL

In [ ]:
#Các bảng cho báo cáo hàng ngày
data_1.to_sql('data_1', cts_engine, if_exists='replace', index=False)
data_2.to_sql('data_2', cts_engine, if_exists='replace', index=False)
data_3.to_sql('data_3', cts_engine, if_exists='replace', index=False)
data_4.to_sql('data_4', cts_engine, if_exists='replace', index=False)
data_5.to_sql('data_5', cts_engine, if_exists='replace', index=False)
data_6.to_sql('data_6', cts_engine, if_exists='replace', index=False)
data_7.to_sql('data_7', cts_engine, if_exists='replace', index=False)
data_8.to_sql('data_8', cts_engine, if_exists='replace', index=False)
data_9.to_sql('data_9', cts_engine, if_exists='replace', index=False)
data_10.to_sql('data_10', cts_engine, if_exists='replace', index=False)
data_11.to_sql('data_11', cts_engine, if_exists='replace', index=False)
data_12.to_sql('data_12', cts_engine, if_exists='replace', index=False)
data_13.to_sql('data_13', cts_engine, if_exists='replace', index=False)
data_14.to_sql('data_14', cts_engine, if_exists='replace', index=False)
time.sleep(120)